## 1. Deep Learning on Tabular Data

In [1]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam,SGD
import copy
import spacy
import re
from sklearn.model_selection import KFold
from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.filterwarnings('ignore')

In [2]:
#reading data
df = pd.read_csv('bank-additional-full.csv',delimiter=';')

Since our label is yes or no, let's make them 0 and 1

In [3]:
#function to map labels
def label_corrector(x):
    if x == 'no':
        return 0
    else:
        return 1

In [4]:
#correcting labels
df['y']=df['y'].apply(label_corrector)

In [5]:
#Pytorch Custom Dataset class which will be feed into data loader
class TabularData(Dataset):
    def __init__ (self, num_cols, cat_cols, target):
        super(TabularData,self).__init__() #initialize the parent class
        self.num_cols = num_cols #numeric columns
        self.cat_cols = cat_cols #categoric columns
        self.y = target
        
    def __len__ (self):
        return self.num_cols.shape[0]
        
    def __getitem__ (self,index):
        return self.num_cols[index,:], self.cat_cols[index,:], self.y[index] #returns numeric col, cat col and target

In [6]:
#class for preprocessing of the data this includes, automatically filtering numeric and cat columns
#numeric columns are standardized and categoric columns are label encoded
class Preprocess:
    def __init__(self, data):
        super(Preprocess,self).__init__()
        self.data = data    
        
    def separate_columns(self,data):
        cat_cols = []
        num_cols = []
        for col in data.columns:
            if data[col].dtype == 'O':
                cat_cols.append(col)
            else:
                num_cols.append(col)
                
        cat_cols = data[cat_cols]
        num_cols = data[num_cols]
        return num_cols, cat_cols
        
    def fit(self):
        num_cols, cat_cols = self.separate_columns(self.data)
        self.scalar = StandardScaler() #standardizing numeric columns
        self.scalar.fit(num_cols)
        self.label_encoder = {} #each column will have its own encoder
        for c in cat_cols.columns:
            encoder = LabelEncoder() #encoding categoric column
            encoder.fit(cat_cols[c].ravel())
            self.label_encoder[c] = encoder
            
    def transform(self,x):
        num_cols, cat_cols = self.separate_columns(x)
        num_cols = pd.DataFrame(self.scalar.transform(num_cols),columns=num_cols.columns,index = cat_cols.index)
        for c in cat_cols.columns:
            encoder = self.label_encoder[c]
            cat_cols[c] = encoder.transform(cat_cols[c].ravel())
        return num_cols, cat_cols
    
    #to know number of unique labels in each categoric column
    def func(self, cat_cols):
        unique_cat_features = [cat_cols.iloc[:,i].nunique() for i in range(cat_cols.shape[1])]
        return unique_cat_features

In [7]:
#train test split with 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.2, random_state=42)

In [8]:
#Preprocessing the training data into a format which our network accepts
P = Preprocess(X_train)
P.fit()

In [9]:
X_train_num, X_train_cat = P.transform(X_train)

In [10]:
#list containing number of unique categories in each column
unique_features_list = P.func(X_train_cat)

In [11]:
#making the training dataset and dataloader
train_data = TabularData(X_train_num.values,X_train_cat.values,y_train.values)
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

In [12]:
#Neural Network Class
class NNetwork(nn.Module):
    def __init__ (self, num_cols, cat_cols, embed_dim, unique_features_list, fc_layers, out_neurons = 1):
        super(NNetwork,self).__init__()
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.embed_dim = embed_dim
        self.unique_cat_features = unique_features_list
        
        #one embedding layer for each categories (it is equivalent to one big embedding layer)
        self.embed_layers = nn.ModuleList([nn.Embedding(c,self.embed_dim) for c in self.unique_cat_features])
        
        #dynamically adding fully connecter layers
        self.fc_neurons = [self.num_cols.shape[1] + len(self.unique_cat_features)*embed_dim] + fc_layers
        self.fcs = nn.Sequential(*[nn.Sequential(nn.Linear(self.fc_neurons[i],self.fc_neurons[i+1]),nn.ReLU()) for i in range(len(self.fc_neurons)-1)])
        
        #output layer
        self.out_layer = nn.Linear(fc_layers[-1],out_neurons)
        
    def forward(self, num_cols, cat_cols):
        #for cat features
        embed_output = []
        for i,el in enumerate(self.embed_layers):
            embed_output.append(el(cat_cols[:,i]))
    
        embed_output = torch.cat(embed_output,1)
        
        #for concatenating cat features with numeric features
        inp = torch.cat([embed_output,num_cols],axis=1)
        
        #for fc layers
        out = self.fcs(inp)
        return self.out_layer(out)

In [13]:
##Implementation of Early Stopping Adopted from https://github.com/Bjarten/early-stopping-pytorch
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [15]:
#trainer function to train the model with early stopping

def trainer(model, n_epochs, train_loader, valid_loader, test_loader, tensorboard, criterion, optimizer, patience):
    
    # to track the training loss and accuracy as the model trains
    train_losses, train_accuracy = [], []
    # to track the validation loss and accuracy as the model trains
    valid_losses, valid_accuracy = [], []
    # to track the test loss and accuracy as the model trains
    test_losses, test_accuracy = [], []
    # to track the average training loss and accuracy per epoch as the model trains
    avg_train_losses, avg_train_accuracies = [],[]
    # to track the average validation loss and accuracy per epoch as the model trains
    avg_valid_losses, avg_valid_accuracies = [], []
    # to track the average test loss and accuracy per epoch as the model trains
    avg_test_losses, avg_test_accuracies = [], []
    
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience, verbose=True)
    step = 0
    for epoch in range(1, n_epochs + 1):
        
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        for nc, cc, target in train_loader:
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(nc,cc)
            # calculate the loss
            loss = criterion(output, target.unsqueeze(1).float())
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # record training loss and accuracy
            train_losses.append(loss.item())
            train_accuracy.append(accuracy_score(target,torch.sigmoid(output)>0.5))

        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        for nc, cc, target in valid_loader:
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(nc,cc)
            # calculate the loss
            loss = criterion(output, target.unsqueeze(1).float())
            # record validation loss and accuracy
            valid_losses.append(loss.item())
            valid_accuracy.append(accuracy_score(target,torch.sigmoid(output)>0.5))

        if test_loader:
            ######################    
            # test the model #
            ######################
            model.eval() # prep model for evaluation
            for nc, cc, target in test_loader:
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model(nc,cc)
                # calculate the loss
                loss = criterion(output, target.unsqueeze(1).float())
                # record validation loss and accuracy
                test_losses.append(loss.item())
                test_accuracy.append(accuracy_score(target,torch.sigmoid(output)>0.5))
                
            test_loss = np.average(test_losses)
            test_accuracy = np.average(test_accuracy)
            avg_test_losses.append(test_loss)
            avg_test_accuracies.append(test_accuracy)
             
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        train_accuracy = np.average(train_accuracy)
        valid_accuracy = np.average(valid_accuracy)
        
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        avg_train_accuracies.append(train_accuracy)
        avg_valid_accuracies.append(valid_accuracy)
        
        if tensorboard:
            #writing the epoch loss for training, validation and testing set in tensorboard
            writer_obj, writer_text = tensorboard
            writer_obj.add_scalar(writer_text+' Training Loss', train_loss, global_step=step)
            writer_obj.add_scalar(writer_text+' Training Accuracy', train_accuracy, global_step=step)
            writer_obj.add_scalar(writer_text+' Validation Loss', valid_loss, global_step=step)
            print(valid_accuracy)
            writer_obj.add_scalar(writer_text+' Validation Accuracy', valid_accuracy, global_step=step)
            writer_obj.add_scalar(writer_text+' Testing Loss', test_loss, global_step=step)
            writer_obj.add_scalar(writer_text+' Testing Accuracy', test_accuracy, global_step=step)
            step+=1
        
        epoch_len = len(str(n_epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        
        print(print_msg+'\n')
        
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        train_accuracy = []
        valid_accuracy = []
        test_losses = []
        test_accuracy = []

        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, model)
        if early_stopping.early_stop:
            print("Early stopped..")
            break
    return model, avg_train_losses, avg_valid_losses, avg_train_accuracies, avg_valid_accuracies, avg_test_losses, avg_test_accuracies

In [16]:
## Hyperparams
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #this ensures to train on GPU if avialble
NUM_EPOCHS = 50 #number of epochs, we can set it to any high number because we are using early stopping anyways
PATIENCE = 6 #patience value for early stopping
K = 3

In [17]:
#function to do KFold Cross Validation on the given set of hyperparams
def KFoldValidation(k, dataset, batch_size, ed, unique_features_list, fc, lr, n_epochs, criterion, optimizer):
    model = NNetwork(X_train_num,X_train_cat,ed,unique_features_list,fc).double().to(DEVICE)
    opt_name = 'Adam' if optimizer == Adam else 'SGD'
    
    #for logging results in tensorboard
    writer = SummaryWriter(f'runs/torch/batch {batch_size} embed_dim {ed} num_lin_layer {fc} optimizer {opt_name} lr {lr}')
    
    splits=KFold(n_splits=k,shuffle=True,random_state=42)
    history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}
    print(f'HyperParams Selected batch_size:{batch_size}, embedding_dim {ed}, num_lin_layer {fc} lr {lr} optimizer {opt_name}')
    for fold, (train_idx, test_idx) in enumerate(splits.split(np.arange(len(dataset)))):
        
        #here the whole training set will be splitted into K folds. if K=3, 2/3 will be used for training and 1/3 for testing
        #out of 2/3 for trianing 10% will be used for validation purposes to facilitate early stopping
        
        model_c = copy.deepcopy(model)
        opt = optimizer(model_c.parameters(),lr)
        
        train_idx, valid_idx = train_test_split(train_idx,test_size=0.1,random_state=101)
        
        #randomly select indexes from training data to be used for trining, valid and testing purposes
        train_sampler = SubsetRandomSampler(train_idx)
        test_sampler = SubsetRandomSampler(test_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)
        
        print('Currently at Fold', fold+1)
        writer_text = 'Fold {}'.format(fold + 1)
        tb = (writer,writer_text)
        
        #the training, validation and testing dataloaders
        train_loader = DataLoader(dataset, batch_size, sampler=train_sampler)
        test_loader = DataLoader(dataset, batch_size, sampler=test_sampler)
        valid_loader = DataLoader(dataset, batch_size, sampler=valid_sampler)
        
        #training the network
        curr_model, avg_tr_loss, avg_va_loss, avg_tr_acc, avg_va_acc, avg_ts_loss, avg_ts_acc  = \
        trainer(model_c, n_epochs, train_loader, valid_loader, test_loader, tb, criterion, optimizer=opt, patience=PATIENCE)
        
        history['train_loss'].append(np.average(avg_tr_loss))
        history['test_loss'].append(np.average(avg_ts_loss))
        history['train_acc'].append(np.average(avg_tr_acc))
        history['test_acc'].append(np.average(avg_ts_acc))
        
        del opt, model_c #to avoid reusing old optimizers and loss
    
    #calculate the average trianing and testing loss over all folds
    avg_train_loss = np.mean(history['train_loss'])
    avg_test_loss = np.mean(history['test_loss'])
    avg_train_acc = np.mean(history['train_acc'])
    avg_test_acc = np.mean(history['test_acc'])
    
    #logging the the K-fold loss and accuracy in tensorboard agaisnt the hyperparameters
    writer.add_hparams({'lr':lr, 'batch_size':batch_size, 'embed_dim':ed, 'num_hidden_neurons': int(np.sum(fc))}, 
                       {'loss': avg_test_loss, 'accuracy':avg_test_acc})
    
    return curr_model, avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc, writer

In [18]:
#function to randomly select/subsample hyperparameters to be used for training
def random_select(config):
    rand_config={}
    for k,v in config.items():
        m = len(v)
        r_val = v[np.random.randint(0,m)]
        rand_config[k] = r_val
    return rand_config

In [19]:
#function to do the K- fold training of the network based on the randomly picked hyper params
def training_function(config):
    rand_config = random_select(config)
    batch_size = rand_config["batch_size"]
    ed = rand_config["ed"]
    fc = rand_config["fc"]
    opt = rand_config["opt"]
    lr = rand_config["lr"]
    model = NNetwork(X_train_num,X_train_cat,ed,unique_features_list,fc).double().to(DEVICE)
    criterion = nn.BCEWithLogitsLoss()
    
    model, _, test_loss, _, _, writer_obj = KFoldValidation(K, train_data, batch_size, ed, unique_features_list, 
                                         fc, lr, NUM_EPOCHS, criterion, opt)
    return test_loss, rand_config, model, writer_obj

In [20]:
#main function which do the K-fold training by radomly selecting hyperparms from the given grid
def run_(config,num_trials):
    min_loss = np.inf #to track the minimum loss
    bestp = None #to track the best params
    writer=None #writer object of best params (this is used for drawing the model later)
    for _ in range(num_trials):
        test_loss, rand_config, model, writer_obj = training_function(config)
        print(test_loss)
        if test_loss < min_loss:
            min_loss = test_loss
            bestp = rand_config
            best_model = model
            writer=writer_obj
    return min_loss, bestp, best_model, writer

In [21]:
## search_space of params to be used
search_space = {
    "batch_size":[64,256,1024],
    "ed":[5,8,10,15,20],
    "fc":[[60,30,20],[30,20,10],[25,25],[40,15],[80,50,30],[50,30,20],[20]],
    "opt":[Adam, SGD],
    "lr": [0.1,0.01,0.2,3e-4,1e-4],
}

#running the function to log the k-fold training results in tensorboard by selecting the random hyperparams
min_loss, best_p, best_model, writer = run_(search_space,10)

HyperParams Selected batch_size:64, embedding_dim 5, num_lin_layer [40, 15] lr 0.2 optimizer Adam
Currently at Fold 1
0.8941539115646259
[ 1/50] train_loss: 0.28091 valid_loss: 0.21018

Validation loss decreased (inf --> 0.210182).  Saving model ...
0.8905399659863946
[ 2/50] train_loss: 0.22612 valid_loss: 0.22497

EarlyStopping counter: 1 out of 6
0.8932397959183673
[ 3/50] train_loss: 0.31946 valid_loss: 0.34399

EarlyStopping counter: 2 out of 6
0.8932397959183673
[ 4/50] train_loss: 0.35504 valid_loss: 0.34272

EarlyStopping counter: 3 out of 6
0.8932397959183673
[ 5/50] train_loss: 0.35514 valid_loss: 0.33970

EarlyStopping counter: 4 out of 6
0.8932397959183673
[ 6/50] train_loss: 0.35687 valid_loss: 0.34407

EarlyStopping counter: 5 out of 6
0.8923256802721089
[ 7/50] train_loss: 0.35533 valid_loss: 0.34445

EarlyStopping counter: 6 out of 6
Early stopped..
Currently at Fold 2
0.8927933673469388
[ 1/50] train_loss: 0.28235 valid_loss: 0.20664

Validation loss decreased (inf -->

0.9209049916107382
[13/50] train_loss: 0.19729 valid_loss: 0.18771

Validation loss decreased (0.196842 --> 0.187715).  Saving model ...
0.9247282228747203
[14/50] train_loss: 0.19399 valid_loss: 0.18138

Validation loss decreased (0.187715 --> 0.181377).  Saving model ...
0.924402702041387
[15/50] train_loss: 0.19312 valid_loss: 0.17783

Validation loss decreased (0.181377 --> 0.177827).  Saving model ...
0.9180583228467561
[16/50] train_loss: 0.19252 valid_loss: 0.19561

EarlyStopping counter: 1 out of 6
0.908458735318792
[17/50] train_loss: 0.19053 valid_loss: 0.18704

EarlyStopping counter: 2 out of 6
0.8995932081935122
[18/50] train_loss: 0.19127 valid_loss: 0.20774

EarlyStopping counter: 3 out of 6
0.9148446238814317
[19/50] train_loss: 0.19063 valid_loss: 0.18482

EarlyStopping counter: 4 out of 6
0.9007707634228188
[20/50] train_loss: 0.19114 valid_loss: 0.22922

EarlyStopping counter: 5 out of 6
0.914193582214765
[21/50] train_loss: 0.19133 valid_loss: 0.18573

EarlyStopping 

0.8945079464951529
[20/50] train_loss: 0.46270 valid_loss: 0.44978

Validation loss decreased (0.460679 --> 0.449781).  Saving model ...
0.8957546840044743
[21/50] train_loss: 0.45304 valid_loss: 0.43885

Validation loss decreased (0.449781 --> 0.438854).  Saving model ...
0.8941962621178224
[22/50] train_loss: 0.44407 valid_loss: 0.43147

Validation loss decreased (0.438854 --> 0.431465).  Saving model ...
0.8904560495898584
[23/50] train_loss: 0.43578 valid_loss: 0.42603

Validation loss decreased (0.431465 --> 0.426030).  Saving model ...
0.8941962621178224
[24/50] train_loss: 0.42686 valid_loss: 0.41429

Validation loss decreased (0.426030 --> 0.414291).  Saving model ...
0.8935728933631617
[25/50] train_loss: 0.41970 valid_loss: 0.40708

Validation loss decreased (0.414291 --> 0.407084).  Saving model ...
0.8941962621178224
[26/50] train_loss: 0.41364 valid_loss: 0.39938

Validation loss decreased (0.407084 --> 0.399384).  Saving model ...
0.8926378402311708
[27/50] train_loss: 0.

0.892203812453393
[32/50] train_loss: 0.38319 valid_loss: 0.37422

Validation loss decreased (0.376689 --> 0.374218).  Saving model ...
0.8946972874720358
[33/50] train_loss: 0.37917 valid_loss: 0.36703

Validation loss decreased (0.374218 --> 0.367033).  Saving model ...
0.8934505499627144
[34/50] train_loss: 0.37545 valid_loss: 0.36490

Validation loss decreased (0.367033 --> 0.364895).  Saving model ...
0.8946972874720358
[35/50] train_loss: 0.37133 valid_loss: 0.35986

Validation loss decreased (0.364895 --> 0.359862).  Saving model ...
0.8928271812080537
[36/50] train_loss: 0.36945 valid_loss: 0.36000

EarlyStopping counter: 1 out of 6
0.8937622343400446
[37/50] train_loss: 0.36703 valid_loss: 0.35597

Validation loss decreased (0.359862 --> 0.355970).  Saving model ...
0.8953206562266965
[38/50] train_loss: 0.36451 valid_loss: 0.35073

Validation loss decreased (0.355970 --> 0.350728).  Saving model ...
0.8937622343400446
[39/50] train_loss: 0.36248 valid_loss: 0.35142

EarlyStop

0.8840242822520508
[45/50] train_loss: 0.34749 valid_loss: 0.35185

EarlyStopping counter: 1 out of 6
0.8846476510067114
[46/50] train_loss: 0.34484 valid_loss: 0.34910

Validation loss decreased (0.351337 --> 0.349100).  Saving model ...
0.8849593353840418
[47/50] train_loss: 0.34385 valid_loss: 0.34795

Validation loss decreased (0.349100 --> 0.347946).  Saving model ...
0.8849593353840418
[48/50] train_loss: 0.34391 valid_loss: 0.34674

Validation loss decreased (0.347946 --> 0.346736).  Saving model ...
0.8846476510067114
[49/50] train_loss: 0.34289 valid_loss: 0.34693

EarlyStopping counter: 1 out of 6
0.8846476510067114
[50/50] train_loss: 0.34204 valid_loss: 0.34518

Validation loss decreased (0.346736 --> 0.345183).  Saving model ...
0.44877004252616753
HyperParams Selected batch_size:64, embedding_dim 8, num_lin_layer [60, 30, 20] lr 0.01 optimizer SGD
Currently at Fold 1
0.8950680272108844
[ 1/50] train_loss: 0.44768 valid_loss: 0.31355

Validation loss decreased (inf --> 0.3

...

0.8957546840044743
[42/50] train_loss: 0.50889 valid_loss: 0.50214

Validation loss decreased (0.505832 --> 0.502145).  Saving model ...
0.8945079464951529
[43/50] train_loss: 0.50613 valid_loss: 0.50028

Validation loss decreased (0.502145 --> 0.500278).  Saving model ...
0.8941962621178224
[44/50] train_loss: 0.50350 valid_loss: 0.49793

Validation loss decreased (0.500278 --> 0.497931).  Saving model ...
0.8960663683818046
[45/50] train_loss: 0.50123 valid_loss: 0.49430

Validation loss decreased (0.497931 --> 0.494304).  Saving model ...
0.8945079464951529
[46/50] train_loss: 0.49856 valid_loss: 0.49263

Validation loss decreased (0.494304 --> 0.492628).  Saving model ...
0.8954429996271439
[47/50] train_loss: 0.49579 valid_loss: 0.48948

Validation loss decreased (0.492628 --> 0.489477).  Saving model ...
0.8957546840044743
[48/50] train_loss: 0.49372 valid_loss: 0.48680

Validation loss decreased (0.489477 --> 0.486798).  Saving model ...
0.8932612089858315
[49/50] train_loss: 0.

0.8770128402311708
[ 2/50] train_loss: 0.63965 valid_loss: 0.63823

Validation loss decreased (0.642819 --> 0.638225).  Saving model ...
0.8793053225205071
[ 3/50] train_loss: 0.63477 valid_loss: 0.63349

Validation loss decreased (0.638225 --> 0.633490).  Saving model ...
0.882410514541387
[ 4/50] train_loss: 0.62996 valid_loss: 0.62874

Validation loss decreased (0.633490 --> 0.628744).  Saving model ...
0.8813531180089486
[ 5/50] train_loss: 0.62543 valid_loss: 0.62445

Validation loss decreased (0.628744 --> 0.624449).  Saving model ...
0.8829115398956003
[ 6/50] train_loss: 0.62097 valid_loss: 0.61966

Validation loss decreased (0.624449 --> 0.619663).  Saving model ...
0.8826552013422818
[ 7/50] train_loss: 0.61660 valid_loss: 0.61602

Validation loss decreased (0.619663 --> 0.616016).  Saving model ...
0.8823435169649515
[ 8/50] train_loss: 0.61252 valid_loss: 0.61190

Validation loss decreased (0.616016 --> 0.611903).  Saving model ...
0.8826552013422818
[ 9/50] train_loss: 0.6

0.8859213331935122
[11/50] train_loss: 0.36729 valid_loss: 0.35055

Validation loss decreased (0.370627 --> 0.350553).  Saving model ...
0.8935677957214766
[12/50] train_loss: 0.34451 valid_loss: 0.32093

Validation loss decreased (0.350553 --> 0.320932).  Saving model ...
0.8973910269854586
[13/50] train_loss: 0.33242 valid_loss: 0.30273

Validation loss decreased (0.320932 --> 0.302728).  Saving model ...
0.8935677957214766
[14/50] train_loss: 0.32298 valid_loss: 0.30458

EarlyStopping counter: 1 out of 6
0.8897445644574944
[15/50] train_loss: 0.31741 valid_loss: 0.30461

EarlyStopping counter: 2 out of 6
0.9012142582494408
[16/50] train_loss: 0.31246 valid_loss: 0.28410

Validation loss decreased (0.302728 --> 0.284098).  Saving model ...
0.8932422748881432
[17/50] train_loss: 0.30525 valid_loss: 0.28853

EarlyStopping counter: 1 out of 6
0.8932837842561522
[18/50] train_loss: 0.30211 valid_loss: 0.27977

Validation loss decreased (0.284098 --> 0.279766).  Saving model ...
0.8997526

0.9270483780760627
[40/50] train_loss: 0.22784 valid_loss: 0.19702

EarlyStopping counter: 1 out of 6
0.9159042051174496
[41/50] train_loss: 0.22566 valid_loss: 0.20419

EarlyStopping counter: 2 out of 6
0.9216390520134228
[42/50] train_loss: 0.22187 valid_loss: 0.19929

EarlyStopping counter: 3 out of 6
0.9082577425894854
[43/50] train_loss: 0.22247 valid_loss: 0.21310

EarlyStopping counter: 4 out of 6
0.914318110318792
[44/50] train_loss: 0.21971 valid_loss: 0.20078

EarlyStopping counter: 5 out of 6
0.925462283277405
[45/50] train_loss: 0.21971 valid_loss: 0.19088

EarlyStopping counter: 6 out of 6
Early stopped..
Currently at Fold 3
0.8749366435961968
[ 1/50] train_loss: 0.67535 valid_loss: 0.66184

Validation loss decreased (inf --> 0.661844).  Saving model ...
0.8787598748601789
[ 2/50] train_loss: 0.65118 valid_loss: 0.64323

Validation loss decreased (0.661844 --> 0.643226).  Saving model ...
0.8730250279642058
[ 3/50] train_loss: 0.63422 valid_loss: 0.62793

Validation loss d

0.9156037414965985
[16/50] train_loss: 0.16709 valid_loss: 0.18806

EarlyStopping counter: 3 out of 6
0.9151360544217687
[17/50] train_loss: 0.16604 valid_loss: 0.18696

EarlyStopping counter: 4 out of 6
0.9128826530612245
[18/50] train_loss: 0.16515 valid_loss: 0.18595

EarlyStopping counter: 5 out of 6
0.9137755102040817
[19/50] train_loss: 0.16403 valid_loss: 0.19252

EarlyStopping counter: 6 out of 6
Early stopped..
Currently at Fold 2
0.9151147959183673
[ 1/50] train_loss: 0.22230 valid_loss: 0.17804

Validation loss decreased (inf --> 0.178038).  Saving model ...
0.9137755102040817
[ 2/50] train_loss: 0.19398 valid_loss: 0.17664

Validation loss decreased (0.178038 --> 0.176644).  Saving model ...
0.913329081632653
[ 3/50] train_loss: 0.18887 valid_loss: 0.17513

Validation loss decreased (0.176644 --> 0.175125).  Saving model ...
0.9120323129251701
[ 4/50] train_loss: 0.18715 valid_loss: 0.17007

Validation loss decreased (0.175125 --> 0.170067).  Saving model ...
0.917368197278

In [22]:
#for drawing the schemetic of the model
dataiter = iter(train_dataloader)
nc, cc, t = next(dataiter)
writer.add_graph(best_model, (nc,cc))
writer.close()
#printing the best loss, params and model
print('The best loss is', min_loss)
print('The best params are', best_p)
print('The best model is', best_model)

The best loss is 0.18903505291088588
The best params are {'batch_size': 64, 'ed': 8, 'fc': [60, 30, 20], 'opt': <class 'torch.optim.adam.Adam'>, 'lr': 0.01}
The best model is NNetwork(
  (embed_layers): ModuleList(
    (0): Embedding(12, 8)
    (1): Embedding(4, 8)
    (2): Embedding(8, 8)
    (3): Embedding(3, 8)
    (4): Embedding(3, 8)
    (5): Embedding(3, 8)
    (6): Embedding(2, 8)
    (7): Embedding(10, 8)
    (8): Embedding(5, 8)
    (9): Embedding(3, 8)
  )
  (fcs): Sequential(
    (0): Sequential(
      (0): Linear(in_features=90, out_features=60, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=60, out_features=30, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=30, out_features=20, bias=True)
      (1): ReLU()
    )
  )
  (out_layer): Linear(in_features=20, out_features=1, bias=True)
)


The best performing model is one trained with Adam optimizer with LR 0.01 Embedding dimension of embedding layer as 8 and three fully connected layer with 60,30,20 neurons each.

Retraining the model on the complete training data with 10% used for validation and testing on the initial 20% that we separated out.

In [23]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=101)
X_valid_num, X_valid_cat = P.transform(X_valid)
valid_data = TabularData(X_valid_num.values,X_valid_cat.values,y_valid.values)
valid_dataloader = DataLoader(valid_data, batch_size=64, shuffle=True)

In [24]:
X_test_num, X_test_cat = P.transform(X_test)
test_data = TabularData(X_test_num.values,X_test_cat.values,y_test.values)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [27]:
optimi = Adam(best_model.parameters(),0.01)
final_model_model, avg_tr_loss, avg_va_loss, avg_tr_acc, avg_va_acc, avg_ts_loss, avg_ts_acc  = \
trainer(best_model, NUM_EPOCHS, train_dataloader, valid_dataloader, test_dataloader, None, nn.BCEWithLogitsLoss(), optimi, patience=PATIENCE)

[ 1/50] train_loss: 0.17633 valid_loss: 0.17114

Validation loss decreased (inf --> 0.171142).  Saving model ...
[ 2/50] train_loss: 0.17303 valid_loss: 0.16982

Validation loss decreased (0.171142 --> 0.169817).  Saving model ...
[ 3/50] train_loss: 0.17136 valid_loss: 0.16897

Validation loss decreased (0.169817 --> 0.168972).  Saving model ...
[ 4/50] train_loss: 0.16936 valid_loss: 0.16216

Validation loss decreased (0.168972 --> 0.162156).  Saving model ...
[ 5/50] train_loss: 0.16753 valid_loss: 0.16432

EarlyStopping counter: 1 out of 6
[ 6/50] train_loss: 0.16605 valid_loss: 0.16306

EarlyStopping counter: 2 out of 6
[ 7/50] train_loss: 0.16525 valid_loss: 0.16376

EarlyStopping counter: 3 out of 6
[ 8/50] train_loss: 0.16432 valid_loss: 0.16104

Validation loss decreased (0.162156 --> 0.161041).  Saving model ...
[ 9/50] train_loss: 0.16451 valid_loss: 0.16700

EarlyStopping counter: 1 out of 6
[10/50] train_loss: 0.16207 valid_loss: 0.15933

Validation loss decreased (0.16104

In [32]:
print('The final test Accuracy of the NNeural Network model on test set is', avg_ts_acc[-1])

The final test Accuracy of the NNeural Network model on test set is 0.9073032524435457


Comparision with CatBoost of our model

In [34]:
import catboost as cb

In [41]:
def get_categorical_indicies(X):
    cats = []
    for col in X.columns:
        if X[col].dtype == 'O':
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(X.columns.get_loc(col))
    return cat_indicies

In [44]:
categorical_indicies = get_categorical_indicies(X_train)

In [45]:
train_dataset = cb.Pool(X_train,y_train, cat_features=categorical_indicies)
test_dataset = cb.Pool(X_test,y_test, cat_features=categorical_indicies)

In [46]:
model = cb.CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5,],
        'iterations': [50, 100, 150]}

In [48]:
model.grid_search(grid,train_dataset)

0:	learn: 0.9024616	test: 0.9047378	best: 0.9047378 (0)	total: 190ms	remaining: 9.3s
1:	learn: 0.9033468	test: 0.9065925	best: 0.9065925 (1)	total: 235ms	remaining: 5.64s
2:	learn: 0.9058338	test: 0.9077727	best: 0.9077727 (2)	total: 290ms	remaining: 4.54s
3:	learn: 0.9044428	test: 0.9060867	best: 0.9077727 (2)	total: 348ms	remaining: 4s
4:	learn: 0.9055808	test: 0.9079413	best: 0.9079413 (4)	total: 405ms	remaining: 3.65s
5:	learn: 0.9040634	test: 0.9069297	best: 0.9079413 (4)	total: 474ms	remaining: 3.48s
6:	learn: 0.9057916	test: 0.9094588	best: 0.9094588 (6)	total: 527ms	remaining: 3.24s
7:	learn: 0.9066346	test: 0.9092902	best: 0.9094588 (6)	total: 593ms	remaining: 3.12s
8:	learn: 0.9047378	test: 0.9084471	best: 0.9094588 (6)	total: 670ms	remaining: 3.05s
9:	learn: 0.9054965	test: 0.9091216	best: 0.9094588 (6)	total: 746ms	remaining: 2.98s
10:	learn: 0.9050329	test: 0.9084471	best: 0.9094588 (6)	total: 796ms	remaining: 2.82s
11:	learn: 0.9052436	test: 0.9089530	best: 0.9094588 (6)	

46:	learn: 0.9160765	test: 0.9194065	best: 0.9194065 (46)	total: 3.21s	remaining: 205ms
47:	learn: 0.9160765	test: 0.9192379	best: 0.9194065 (46)	total: 3.27s	remaining: 136ms
48:	learn: 0.9162452	test: 0.9204181	best: 0.9204181 (48)	total: 3.33s	remaining: 68ms
49:	learn: 0.9164981	test: 0.9207554	best: 0.9207554 (49)	total: 3.4s	remaining: 0us

bestTest = 0.9207553532
bestIteration = 49

1:	loss: 0.9207554	best: 0.9207554 (1)	total: 6.87s	remaining: 2m 58s
0:	learn: 0.9024616	test: 0.9047378	best: 0.9047378 (0)	total: 84.2ms	remaining: 4.13s
1:	learn: 0.9030096	test: 0.9054122	best: 0.9054122 (1)	total: 180ms	remaining: 4.31s
2:	learn: 0.9042742	test: 0.9072669	best: 0.9072669 (2)	total: 242ms	remaining: 3.79s
3:	learn: 0.9043163	test: 0.9070983	best: 0.9072669 (2)	total: 308ms	remaining: 3.54s
4:	learn: 0.9044006	test: 0.9070983	best: 0.9072669 (2)	total: 409ms	remaining: 3.68s
5:	learn: 0.9057073	test: 0.9076041	best: 0.9076041 (5)	total: 484ms	remaining: 3.55s
6:	learn: 0.9063396	

39:	learn: 0.9148120	test: 0.9185635	best: 0.9185635 (39)	total: 2.57s	remaining: 643ms
40:	learn: 0.9150228	test: 0.9187321	best: 0.9187321 (40)	total: 2.66s	remaining: 583ms
41:	learn: 0.9151071	test: 0.9185635	best: 0.9187321 (40)	total: 2.74s	remaining: 522ms
42:	learn: 0.9147699	test: 0.9187321	best: 0.9187321 (40)	total: 2.83s	remaining: 460ms
43:	learn: 0.9150649	test: 0.9182263	best: 0.9187321 (40)	total: 2.92s	remaining: 398ms
44:	learn: 0.9152335	test: 0.9189007	best: 0.9189007 (44)	total: 3s	remaining: 333ms
45:	learn: 0.9156550	test: 0.9190693	best: 0.9190693 (45)	total: 3.08s	remaining: 268ms
46:	learn: 0.9155707	test: 0.9189007	best: 0.9190693 (45)	total: 3.15s	remaining: 201ms
47:	learn: 0.9159922	test: 0.9194065	best: 0.9194065 (47)	total: 3.22s	remaining: 134ms
48:	learn: 0.9159501	test: 0.9199123	best: 0.9199123 (48)	total: 3.3s	remaining: 67.3ms
49:	learn: 0.9160765	test: 0.9202495	best: 0.9202495 (49)	total: 3.39s	remaining: 0us

bestTest = 0.9202495363
bestIteratio

32:	learn: 0.9141376	test: 0.9192379	best: 0.9192379 (30)	total: 3.26s	remaining: 1.68s
33:	learn: 0.9138425	test: 0.9187321	best: 0.9192379 (30)	total: 3.48s	remaining: 1.64s
34:	learn: 0.9141376	test: 0.9190693	best: 0.9192379 (30)	total: 3.63s	remaining: 1.56s
35:	learn: 0.9145169	test: 0.9190693	best: 0.9192379 (30)	total: 3.8s	remaining: 1.48s
36:	learn: 0.9146012	test: 0.9190693	best: 0.9192379 (30)	total: 3.95s	remaining: 1.39s
37:	learn: 0.9146012	test: 0.9187321	best: 0.9192379 (30)	total: 4.1s	remaining: 1.29s
38:	learn: 0.9147699	test: 0.9187321	best: 0.9192379 (30)	total: 4.22s	remaining: 1.19s
39:	learn: 0.9148120	test: 0.9189007	best: 0.9192379 (30)	total: 4.37s	remaining: 1.09s
40:	learn: 0.9149806	test: 0.9189007	best: 0.9192379 (30)	total: 4.48s	remaining: 984ms
41:	learn: 0.9152335	test: 0.9192379	best: 0.9192379 (30)	total: 4.61s	remaining: 877ms
42:	learn: 0.9152757	test: 0.9194065	best: 0.9194065 (42)	total: 4.72s	remaining: 769ms
43:	learn: 0.9154864	test: 0.91906

76:	learn: 0.9100910	test: 0.9131681	best: 0.9131681 (66)	total: 7.04s	remaining: 2.1s
77:	learn: 0.9093323	test: 0.9126623	best: 0.9131681 (66)	total: 7.13s	remaining: 2.01s
78:	learn: 0.9091637	test: 0.9121565	best: 0.9131681 (66)	total: 7.23s	remaining: 1.92s
79:	learn: 0.9092902	test: 0.9119879	best: 0.9131681 (66)	total: 7.33s	remaining: 1.83s
80:	learn: 0.9097960	test: 0.9124937	best: 0.9131681 (66)	total: 7.43s	remaining: 1.74s
81:	learn: 0.9099646	test: 0.9131681	best: 0.9131681 (66)	total: 7.53s	remaining: 1.65s
82:	learn: 0.9108076	test: 0.9136739	best: 0.9136739 (82)	total: 7.62s	remaining: 1.56s
83:	learn: 0.9111027	test: 0.9136739	best: 0.9136739 (82)	total: 7.7s	remaining: 1.47s
84:	learn: 0.9100910	test: 0.9135053	best: 0.9136739 (82)	total: 7.78s	remaining: 1.37s
85:	learn: 0.9111027	test: 0.9143483	best: 0.9143483 (85)	total: 7.86s	remaining: 1.28s
86:	learn: 0.9112291	test: 0.9143483	best: 0.9143483 (85)	total: 7.93s	remaining: 1.19s
87:	learn: 0.9112713	test: 0.91401

...

110:	learn: 0.9224543	test: 0.9135141	best: 0.9140198 (57)	total: 23.9s	remaining: 8.4s
111:	learn: 0.9226567	test: 0.9134129	best: 0.9140198 (57)	total: 24.1s	remaining: 8.18s
112:	learn: 0.9226061	test: 0.9138175	best: 0.9140198 (57)	total: 24.2s	remaining: 7.94s
113:	learn: 0.9225049	test: 0.9137164	best: 0.9140198 (57)	total: 24.4s	remaining: 7.7s
114:	learn: 0.9226567	test: 0.9139187	best: 0.9140198 (57)	total: 24.6s	remaining: 7.47s
115:	learn: 0.9228084	test: 0.9140198	best: 0.9140198 (57)	total: 24.8s	remaining: 7.26s
116:	learn: 0.9230614	test: 0.9143233	best: 0.9143233 (116)	total: 25.1s	remaining: 7.07s
117:	learn: 0.9229602	test: 0.9145256	best: 0.9145256 (117)	total: 25.3s	remaining: 6.86s
118:	learn: 0.9228590	test: 0.9143233	best: 0.9145256 (117)	total: 25.5s	remaining: 6.64s
119:	learn: 0.9230614	test: 0.9141210	best: 0.9145256 (117)	total: 25.7s	remaining: 6.43s
120:	learn: 0.9232637	test: 0.9141210	best: 0.9145256 (117)	total: 25.9s	remaining: 6.21s
121:	learn: 0.9231

54:	learn: 0.9196257	test: 0.9145169	best: 0.9145169 (54)	total: 9.74s	remaining: 16.8s
55:	learn: 0.9197269	test: 0.9146181	best: 0.9146181 (55)	total: 9.9s	remaining: 16.6s
56:	learn: 0.9195751	test: 0.9147193	best: 0.9147193 (56)	total: 10.1s	remaining: 16.5s
57:	learn: 0.9197269	test: 0.9145169	best: 0.9147193 (56)	total: 10.3s	remaining: 16.3s
58:	learn: 0.9201315	test: 0.9143146	best: 0.9147193 (56)	total: 10.4s	remaining: 16.1s
59:	learn: 0.9202833	test: 0.9145169	best: 0.9147193 (56)	total: 10.6s	remaining: 15.9s
60:	learn: 0.9204350	test: 0.9145169	best: 0.9147193 (56)	total: 10.8s	remaining: 15.7s
61:	learn: 0.9203844	test: 0.9146181	best: 0.9147193 (56)	total: 11s	remaining: 15.6s
62:	learn: 0.9206879	test: 0.9142135	best: 0.9147193 (56)	total: 11.2s	remaining: 15.4s
63:	learn: 0.9207891	test: 0.9143146	best: 0.9147193 (56)	total: 11.3s	remaining: 15.2s
64:	learn: 0.9208397	test: 0.9143146	best: 0.9147193 (56)	total: 11.4s	remaining: 14.9s
65:	learn: 0.9208902	test: 0.914011

148:	learn: 0.9261507	test: 0.9153263	best: 0.9158321 (85)	total: 19.4s	remaining: 130ms
149:	learn: 0.9263531	test: 0.9155286	best: 0.9158321 (85)	total: 19.4s	remaining: 0us

bestTest = 0.9158320688
bestIteration = 85

Training on fold [2/3]
0:	learn: 0.9062263	test: 0.9072238	best: 0.9072238 (0)	total: 76.3ms	remaining: 11.4s
1:	learn: 0.9051641	test: 0.9075273	best: 0.9075273 (1)	total: 162ms	remaining: 12s
2:	learn: 0.9049112	test: 0.9107649	best: 0.9107649 (2)	total: 254ms	remaining: 12.5s
3:	learn: 0.9065803	test: 0.9096520	best: 0.9107649 (2)	total: 339ms	remaining: 12.4s
4:	learn: 0.9040008	test: 0.9087414	best: 0.9107649 (2)	total: 428ms	remaining: 12.4s
5:	learn: 0.9049112	test: 0.9091461	best: 0.9107649 (2)	total: 517ms	remaining: 12.4s
6:	learn: 0.9058722	test: 0.9101578	best: 0.9107649 (2)	total: 609ms	remaining: 12.4s
7:	learn: 0.9070356	test: 0.9105625	best: 0.9107649 (2)	total: 692ms	remaining: 12.3s
8:	learn: 0.9074402	test: 0.9128895	best: 0.9128895 (8)	total: 774ms	

91:	learn: 0.9189217	test: 0.9209834	best: 0.9213881 (80)	total: 8.8s	remaining: 5.55s
92:	learn: 0.9193263	test: 0.9207811	best: 0.9213881 (80)	total: 8.93s	remaining: 5.48s
93:	learn: 0.9195792	test: 0.9206799	best: 0.9213881 (80)	total: 9.1s	remaining: 5.42s
94:	learn: 0.9197309	test: 0.9208822	best: 0.9213881 (80)	total: 9.23s	remaining: 5.34s
95:	learn: 0.9195792	test: 0.9207811	best: 0.9213881 (80)	total: 9.36s	remaining: 5.26s
96:	learn: 0.9199838	test: 0.9205787	best: 0.9213881 (80)	total: 9.5s	remaining: 5.19s
97:	learn: 0.9198827	test: 0.9209834	best: 0.9213881 (80)	total: 9.62s	remaining: 5.1s
98:	learn: 0.9197309	test: 0.9206799	best: 0.9213881 (80)	total: 9.74s	remaining: 5.02s
99:	learn: 0.9198827	test: 0.9207811	best: 0.9213881 (80)	total: 9.85s	remaining: 4.92s
100:	learn: 0.9195286	test: 0.9207811	best: 0.9213881 (80)	total: 9.97s	remaining: 4.83s
101:	learn: 0.9196803	test: 0.9207811	best: 0.9213881 (80)	total: 10.1s	remaining: 4.77s
102:	learn: 0.9197815	test: 0.9207

{'params': {'depth': 4,
  'l2_leaf_reg': 1,
  'iterations': 150,
  'learning_rate': 0.1},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,

In [49]:
pred = model.predict(X_test)

In [52]:
print('The Accuracy of the Catboost model is',accuracy_score(y_test,pred))

The Accuracy of the Catboost model is 0.9187909686817188


## 2. NLP - Word2Vec Model

In [2]:
## Continuous Bag of Word Model
class CBOW(nn.Module):
    def __init__(self, vocab_size, embed_dim, lin_layer_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_dim) #embedding layer
        self.linear = nn.Linear(embed_dim, lin_layer_size) #linear layer
        self.relu = nn.ReLU()
        self.out = nn.Linear(lin_layer_size, vocab_size) #output layer
        
    def forward(self, x):
        embed = self.embeddings(x).mean(1) #take the avergae of all the embeddings in context of a word
        return self.out(self.relu(self.linear(embed)))

In [3]:
#for preprocessing the text, tokenizing it
class PreprocessText:
    def __init__ (self, corpus):
        nlp = spacy.load("en_core_web_sm")
        self.tokenized_text = [token.text for token in nlp(corpus)]
        self.vocab = set(self.tokenized_text)
        self.vocab_size = len(self.vocab)
        self.idx_to_word= {i:w for i,w in enumerate(self.vocab)}
        self.word_to_idx = {w:i for i,w in enumerate(self.vocab)}
        
    def create_dataset(self,n_context_words):
        self.data = []
        for i in range(n_context_words, len(self.tokenized_text) - n_context_words):
            context = []
            for curr_idx in reversed(range(-n_context_words,n_context_words+1)):
                if curr_idx == 0:
                    continue
                context.append(self.word_to_idx[self.tokenized_text[i-curr_idx]])
            
            context.append(self.word_to_idx[self.tokenized_text[i]])
            self.data.append(context)
        self.targets = torch.tensor(self.data)[:,-1]
        self.data = torch.tensor(self.data)[:,:-1]

In [4]:
#Hyperparameter
NUM_CONTEXT_WORDS = 2
BATCH_SIZE = 64
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #this ensures to train on GPU if avialble
NUM_EPOCHS = 50
EMBED_DIM = 100
LIN_LAYER_SIZE = 128
LR = 0.01
PADDING = ' padding ' * NUM_CONTEXT_WORDS

In [5]:
with open('raw_text.txt', "r", encoding="utf-8") as f:
    raw_text = f.read()
    
raw_text = PADDING + raw_text + PADDING
raw_text = re.sub('\W+',' ', raw_text).strip().lower() #for removing pronunciation 

In [6]:
#preprocessing the text corpus
text_corpus = PreprocessText(raw_text)
text_corpus.create_dataset(NUM_CONTEXT_WORDS)

In [7]:
#making the text dataloader
text_dataset = torch.utils.data.TensorDataset(text_corpus.data,text_corpus.targets)
text_dataloader = DataLoader(text_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
VOCAB_SIZE = text_corpus.vocab_size

## Defining Optimizer and Loss
model = CBOW(VOCAB_SIZE, EMBED_DIM, LIN_LAYER_SIZE).double().to(DEVICE) #make the model, send it to GPU and set double precision to avoid errors
optimizer = Adam(model.parameters(),lr=LR) #the adam optimizer is used to train the network
criterion = nn.CrossEntropyLoss() #BCE loss, when loss is applied, sqrt is taken to make it RMSE

In [9]:
# Trainer for training the text
def text_trainer(model, n_epochs, train_loader, criterion, optimizer):
    
    for epoch in range(1, n_epochs + 1):
        epoch_loss = []
        for data, target in train_loader:
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # record training loss and accuracy
            epoch_loss.append(loss.item())
         
        train_loss = np.mean(epoch_loss)
        epoch_len = len(str(n_epochs))
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f}')
        
        print(print_msg+'\n')
        
    return model

In [10]:
trained_model = text_trainer(model, NUM_EPOCHS, text_dataloader, criterion, optimizer)

[ 1/50] train_loss: 4.62118

[ 2/50] train_loss: 3.86027

[ 3/50] train_loss: 3.07001

[ 4/50] train_loss: 2.29802

[ 5/50] train_loss: 1.52846

[ 6/50] train_loss: 0.89494

[ 7/50] train_loss: 0.46776

[ 8/50] train_loss: 0.22740

[ 9/50] train_loss: 0.10398

[10/50] train_loss: 0.04784

[11/50] train_loss: 0.02662

[12/50] train_loss: 0.01527

[13/50] train_loss: 0.00932

[14/50] train_loss: 0.00630

[15/50] train_loss: 0.00459

[16/50] train_loss: 0.00346

[17/50] train_loss: 0.00273

[18/50] train_loss: 0.00225

[19/50] train_loss: 0.00193

[20/50] train_loss: 0.00170

[21/50] train_loss: 0.00152

[22/50] train_loss: 0.00138

[23/50] train_loss: 0.00127

[24/50] train_loss: 0.00119

[25/50] train_loss: 0.00112

[26/50] train_loss: 0.00106

[27/50] train_loss: 0.00102

[28/50] train_loss: 0.00098

[29/50] train_loss: 0.00095

[30/50] train_loss: 0.00092

[31/50] train_loss: 0.00089

[32/50] train_loss: 0.00087

[33/50] train_loss: 0.00085

[34/50] train_loss: 0.00083

[35/50] train_

In [12]:
#create a summary writer
writer = SummaryWriter('runs/text/')
writer.add_embedding(trained_model.embeddings.weight.data, metadata = list(text_corpus.vocab), tag='embeddings')
writer.close()

<img src="PNGS/projections.png" width="750" align="center">

Since the size of our dataset is not huge, the embedding generation is not very accurate for each word. 